In [0]:
from delta.tables import *
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, array, ArrayType, DateType, TimestampType, FloatType
from pyspark.sql.functions import *

In [0]:
df_sales_orders = spark.read                          \
                         .option("header", "true")      \
                         .option("inferSchema", "true") \
                         .csv("/Volumes/my_first_catalog/my_first_schema/my_staging_data/sales_transactions.csv")
display(df_sales_orders)
df_sales_orders.printSchema()

In [0]:
%sql
DROP TABLE IF EXISTS my_first_catalog.my_first_schema.store_orders;

In [0]:
df_sales_orders.write.format("delta").partitionBy("currency").saveAsTable("my_first_catalog.my_first_schema.store_orders")

In [0]:
%%sql		
SELECT * FROM my_first_catalog.my_first_schema.store_orders;

In [0]:
%%sql
DESCRIBE my_first_catalog.my_first_schema.store_orders;

In [0]:
%%sql
DESCRIBE HISTORY my_first_catalog.my_first_schema.store_orders;

In [0]:
%%sql
SELECT * FROM my_first_catalog.my_first_schema.store_orders WHERE order_number=2581; 

In [0]:
%%sql
UPDATE my_first_catalog.my_first_schema.store_orders SET sale_price=90.50 WHERE order_number=2581;

In [0]:
%%sql
SELECT * FROM my_first_catalog.my_first_schema.store_orders WHERE order_number=2581;


In [0]:
%%sql
DESCRIBE HISTORY my_first_catalog.my_first_schema.store_orders;


In [0]:
%%sql
SELECT * FROM my_first_catalog.my_first_schema.store_orders VERSION AS OF 0 WHERE order_number=2581;

In [0]:
%%sql
DELETE FROM my_first_catalog.my_first_schema.store_orders WHERE order_number=2581;
SELECT * FROM my_first_catalog.my_first_schema.store_orders WHERE order_number=2581;


In [0]:
%%sql
SELECT count(*) FROM my_first_catalog.my_first_schema.store_orders;

In [0]:
%%sql
DESCRIBE HISTORY my_first_catalog.my_first_schema.store_orders;

In [0]:
%%sql
RESTORE TABLE my_first_catalog.my_first_schema.store_orders TO VERSION AS OF 1;
SELECT * FROM my_first_catalog.my_first_schema.store_orders WHERE order_number=2581; 

In [0]:
%%sql
DESCRIBE HISTORY my_first_catalog.my_first_schema.store_orders;


In [0]:
%%sql
SELECT count(*) FROM my_first_catalog.my_first_schema.store_orders;

In [0]:
%%sql
SELECT * FROM my_first_catalog.my_first_schema.store_orders WHERE order_number=2501; 

In [0]:
df_sales_orders_CDC1 = spark.read                          \
                         .option("header", "true")      \
                         .option("inferSchema", "true") \
                         .csv("/Volumes/my_first_catalog/my_first_schema/my_staging_data/sales_transactions - CDC1.csv")
df_sales_orders_CDC1 = df_sales_orders_CDC1.withColumn("order_date", to_date(df_sales_orders_CDC1.order_date,  'MM/dd/yyyy'))
display(df_sales_orders_CDC1)
df_sales_orders_CDC1.printSchema()
df_sales_orders_CDC1.createOrReplaceTempView("cdc_store_orders")


In [0]:
spark.sql("SELECT * FROM cdc_store_orders").show()

In [0]:
%sql
MERGE INTO my_first_catalog.my_first_schema.store_orders
USING cdc_store_orders
ON cdc_store_orders.order_number = my_first_catalog.my_first_schema.store_orders.order_number
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED THEN
  INSERT *

In [0]:
%%sql
SELECT * FROM my_first_catalog.my_first_schema.store_orders WHERE order_number=2501; 


In [0]:
%%sql
DESCRIBE HISTORY my_first_catalog.my_first_schema.store_orders;


In [0]:
df_read_data_schema_change = spark.read                           \
                                .option("header", "true")         \
                                .option("inferSchema", "true")    \
                                .csv("/Volumes/my_first_catalog/my_first_schema/my_staging_data/schema_change.csv")
df_read_data_schema_change = df_read_data_schema_change.withColumn("order_date", to_date(df_read_data_schema_change.order_date,  'MM/dd/yyyy'))
display(df_read_data_schema_change)
df_read_data_schema_change.printSchema()
df_read_data_schema_change.createOrReplaceTempView("schema_change_store_orders")

In [0]:
%sql
MERGE WITH SCHEMA EVOLUTION INTO my_first_catalog.my_first_schema.store_orders
USING schema_change_store_orders
ON schema_change_store_orders.order_number = my_first_catalog.my_first_schema.store_orders.order_number
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED THEN
  INSERT *

In [0]:
%%sql
SELECT * FROM my_first_catalog.my_first_schema.store_orders; 